In [1]:
import sys

sys.path.append("../")

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

from recgame.environments import BaseEnvironment
from recgame.recourse import ActionableRecourse, NFeatureRecourse

rng = np.random.default_rng(42)

def data_source_func(n_agents):
    df = pd.DataFrame(rng.random((n_agents, 4)), columns=["a", "b", "c", "d"])
    df["cat_1"] = rng.integers(0, 2, n_agents)
    return df

df = data_source_func(100)
y = rng.integers(0, 2, 100)

lr = LogisticRegression().fit(df, y)

### Environment using ``ActionableRecourse``

In [3]:
# Test an environment
recourse = ActionableRecourse(model=lr, categorical=["cat_1"], threshold=0.6)
recourse.set_actions(df)

environment = BaseEnvironment(
    X=df,
    recourse=recourse,
    data_source_func=data_source_func,
    threshold=10,
    threshold_type="absolute",
    growth_rate=10,
    growth_rate_type="absolute",
    adaptation=.1,
    behavior_function="binary_constant",
)

In [4]:
assert environment.step_ == 0

environment.simulate(6)

assert environment.step_ == 6
assert environment.X_.shape[0] == 100
assert (environment.X_.dtypes == environment.X.dtypes).all()

AssertionError: 

In [6]:
environment.X_.dtypes, environment.X.dtypes

(a        float64
 b        float64
 c        float64
 d        float64
 cat_1    float64
 dtype: object,
 a        float64
 b        float64
 c        float64
 d        float64
 cat_1      int64
 dtype: object)

In [7]:
[environment.get_number_new_agents(i) for i in range(6)]
[environment.get_score_threshold(i) for i in range(6)]
[(i, environment.metadata_[i]["threshold_index"]) for i in range(len(environment.metadata_))]

[(0, 90), (1, 90), (2, 90), (3, 90), (4, 90), (5, 90), (6, 90)]

In [8]:
[(i, environment.metadata_[i]["X"].shape) for i in range(len(environment.metadata_))]

[(0, (100, 5)),
 (1, (100, 5)),
 (2, (100, 5)),
 (3, (100, 5)),
 (4, (100, 5)),
 (5, (100, 5)),
 (6, (100, 5))]

### Environment using ``NFeatureRecourse``

In [9]:
recourse = NFeatureRecourse(model=lr, threshold=0.6, categorical=["cat_1"],)

environment = BaseEnvironment(
    X=df,
    recourse=recourse,
    data_source_func=data_source_func,
    threshold=10,
    threshold_type="absolute",
    growth_rate=11,
    growth_rate_type="absolute",
    adaptation=.1,
    behavior_function="binary_constant",
)

In [10]:
assert environment.step_ == 0

environment.simulate(6)

assert environment.step_ == 6
assert environment.X_.shape[0] == 106
assert (environment.X_.dtypes == environment.X.dtypes).all()

AssertionError: 

In [11]:
[(i, environment.metadata_[i]["growth_k"]) for i in range(len(environment.metadata_))]
[(i, environment.metadata_[i]["outcome"].sum()) for i in range(len(environment.metadata_))]
[(i, environment.metadata_[i]["X"].shape[0]) for i in range(len(environment.metadata_))]

[(0, 100), (1, 101), (2, 102), (3, 103), (4, 104), (5, 105), (6, 106)]

In [12]:
environment.metadata_[2].keys()

dict_keys(['X', 'effort', 'outcome', 'score', 'threshold', 'growth_k', 'threshold_index'])

In [14]:
environment.counterfactual(environment.metadata_[1]["X"])

,a,b,c,d,cat_1
0,0.819242,0.566010,0.943380,0.704678,0.000000
2,0.304148,0.944569,0.700362,0.955181,0.975993
3,0.692460,0.959182,0.534391,0.235083,0.993373
4,0.753057,0.620990,0.970698,0.663702,0.972932
5,0.837597,0.577734,0.970698,0.905956,0.989157
...,...,...,...,...,...
106,0.341029,0.992376,0.601637,0.215722,0.000000
107,0.747511,0.770772,0.780381,0.320757,0.996955
108,0.979571,0.933880,0.352801,0.163272,0.000000
109,0.865590,0.657188,0.830229,0.969176,0.976475
